# FRC 2026 REBUILT - Lob Shooter Analysis
## Team 2491 Engineering Toolkit

This notebook provides interactive tools for analyzing **lob trajectories** for the 2026 FRC game.

### Key Insight: Ball Must DESCEND Into Funnel
The target is a funnel with the opening facing **UP**. Like throwing a ball into a trash can, we need HIGH-ANGLE shots that arc up and come DOWN into the opening.

### Game Constraints
- **Projectile (FUEL):** 5.91" diameter foam ball, ~0.47 lbs
- **Target:** 72" high funnel opening facing UP
- **Robot:** Max 30" height, practical release ~18"
- **Shooting Range:** 24" (2 ft) to 250" (21 ft) - full alliance zone
- **Recommended Velocity:** 34 ft/s (covers full range with steep entry angles)

---
## 1. Setup and Constants

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Polygon
from ipywidgets import interact, interactive, FloatSlider, IntSlider, Dropdown, HBox, VBox
import ipywidgets as widgets
from IPython.display import display, Markdown

# Enable interactive plots
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 11

# Physical constants
G = 386.09  # in/s^2 (gravity in imperial)

# Game piece properties
FUEL_DIAMETER_IN = 5.91
FUEL_MASS_LB = 0.47
FUEL_MASS_KG = FUEL_MASS_LB * 0.4536

# Target properties
TARGET_HEIGHT_IN = 72

# Shooting range
MIN_DISTANCE_IN = 24   # 2 ft minimum
MAX_DISTANCE_IN = 250  # ~21 ft maximum (alliance zone corner)

# Recommended velocity (covers full range with steep entry)
RECOMMENDED_VELOCITY_FPS = 30

# Robot constraints
ROBOT_MAX_HEIGHT_IN = 30
RELEASE_HEIGHT_IN = 18

print("LOB SHOOTER ANALYSIS - Setup Complete")
print("=" * 50)
print(f"Gravity: {G} in/s^2")
print(f"Target height: {TARGET_HEIGHT_IN} in (funnel facing UP)")
print(f"Shooting range: {MIN_DISTANCE_IN}\" to {MAX_DISTANCE_IN}\"")
print(f"Recommended velocity: {RECOMMENDED_VELOCITY_FPS} ft/s")
print("=" * 50)

---
## 2. Understanding Lob Trajectories

### Why Lob Shots?

The target is a **funnel facing UP**. This means:
- Ball must be **DESCENDING** (vy < 0) when it reaches the target height
- Ball arcs **UP** above the target, then falls **DOWN** into the opening
- Higher entry angles = steeper descent = better capture

**Analogy:** Like throwing a ball into a tall trash can - you lob it up so it falls in from above.

### Key Condition
For a valid lob shot, when the ball reaches `y = target_height` at `x = 0`:
- **vy must be NEGATIVE** (ball going down, not up)
- This eliminates "line drive" trajectories that pass through the target while ascending

### Entry Angle Quality
- **IDEAL (60-90°):** Steep descent, high capture rate
- **GOOD (45-60°):** Reliable capture
- **RISKY (<45°):** May bounce off funnel walls

In [ ]:
def find_lob_solution(distance_in, velocity_fps=RECOMMENDED_VELOCITY_FPS, 
                      release_height=RELEASE_HEIGHT_IN, vx_robot=0, vy_robot=0):
    """
    Find the launch angle for a LOB trajectory where ball DESCENDS into target.
    
    CRITICAL: Only returns solutions where vy < 0 at target (ball descending).
    
    Returns: dict with angle, entry_angle, max_height, etc. or None if no solution
    """
    v_shot = velocity_fps * 12  # Convert to in/s
    
    best_solution = None
    best_entry_angle = 0
    
    # Scan from low to high angles
    for theta_deg in np.linspace(5, 89, 2000):
        theta = np.radians(theta_deg)
        
        # Combined initial velocity
        vx0 = v_shot * np.cos(theta) + vx_robot
        vy0 = v_shot * np.sin(theta) + vy_robot
        
        if vx0 <= 0:
            continue
        
        # Time to reach target x-position
        t_hit = distance_in / vx0
        
        # Height and vertical velocity at target
        y_at_target = release_height + vy0 * t_hit - 0.5 * G * t_hit**2
        vy_at_target = vy0 - G * t_hit
        
        # CRITICAL: Ball must be at target height AND DESCENDING
        if abs(y_at_target - TARGET_HEIGHT_IN) < 2.5 and vy_at_target < 0:
            # Entry angle = how steeply ball is descending
            entry_angle = np.degrees(np.arctan2(abs(vy_at_target), vx0))
            
            # Keep solution with steepest entry (best for funnel)
            if entry_angle > best_entry_angle:
                best_entry_angle = entry_angle
                
                # Calculate max height
                t_apex = vy0 / G
                max_height = release_height + vy0 * t_apex - 0.5 * G * t_apex**2
                
                best_solution = {
                    'angle_deg': theta_deg,
                    'entry_angle_deg': entry_angle,
                    't_flight': t_hit,
                    'max_height': max_height,
                    'y_at_target': y_at_target,
                    'vy_at_target': vy_at_target,
                    'vx0': vx0,
                    'vy0': vy0
                }
    
    return best_solution


def calculate_trajectory(distance_in, velocity_fps, theta_deg, release_height=RELEASE_HEIGHT_IN,
                        vx_robot=0, vy_robot=0):
    """Calculate full trajectory points for plotting."""
    v_shot = velocity_fps * 12
    theta = np.radians(theta_deg)
    
    vx0 = v_shot * np.cos(theta) + vx_robot
    vy0 = v_shot * np.sin(theta) + vy_robot
    
    if vx0 <= 0:
        return np.array([0]), np.array([distance_in]), np.array([release_height])
    
    t_max = 3.0  # Max 3 seconds
    t = np.linspace(0, t_max, 500)
    
    x = distance_in - vx0 * t
    y = release_height + vy0 * t - 0.5 * G * t**2
    
    # Keep valid portion (above ground, not past target)
    valid = (x >= -20) & (y >= -5)
    return t[valid], x[valid], y[valid]


# Test the function
print("Testing lob solution finder...")
test_dist = 84  # 7 ft
solution = find_lob_solution(test_dist)

if solution:
    print(f"\nFor {test_dist/12:.0f} ft shot at {RECOMMENDED_VELOCITY_FPS} fps:")
    print(f"  Launch angle: {solution['angle_deg']:.1f} deg")
    print(f"  Entry angle:  {solution['entry_angle_deg']:.1f} deg (ball descending)")
    print(f"  Max height:   {solution['max_height']:.0f} in")
    print(f"  Flight time:  {solution['t_flight']:.2f} s")
else:
    print("No valid lob solution found!")

---
## 3. Interactive Lob Trajectory Visualization

Use the sliders to explore how distance and robot motion affect the lob trajectory.

**Key observations:**
- The ball arcs HIGH above the target, then falls DOWN into the funnel
- Entry angles are steep (60-90 deg) - excellent for funnel capture
- Launch angles are high (70-88 deg) - these are true lob shots

In [ ]:
def plot_lob_trajectory_interactive(distance_ft=7, velocity_fps=30, robot_velocity_fps=0):
    """Interactive lob trajectory plotter."""
    distance_in = distance_ft * 12
    vx_robot = robot_velocity_fps * 12
    
    # Find the lob solution
    solution = find_lob_solution(distance_in, velocity_fps, vx_robot=vx_robot)
    
    # Create figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))
    
    if solution:
        # Calculate trajectory
        t, x, y = calculate_trajectory(distance_in, velocity_fps, solution['angle_deg'],
                                       vx_robot=vx_robot)
        
        # Left plot: Trajectory
        ax1.plot(x, y, 'b-', linewidth=3, label='Lob trajectory')
        ax1.plot(distance_in, RELEASE_HEIGHT_IN, 'ko', markersize=12, label='Release point')
        
        # Mark apex
        ax1.plot(x[np.argmax(y)], np.max(y), 'g^', markersize=12, label=f'Apex: {solution["max_height"]:.0f}"')
        
        # Draw entry arrow showing descent
        ax1.annotate('', xy=(0, TARGET_HEIGHT_IN), xytext=(15, TARGET_HEIGHT_IN + 25),
                    arrowprops=dict(arrowstyle='->', color='red', lw=3))
        ax1.annotate(f'{solution["entry_angle_deg"]:.0f} deg\ndescent', (18, TARGET_HEIGHT_IN + 15),
                    fontsize=11, color='red', fontweight='bold')
    
    # Funnel target (opening faces UP)
    ax1.plot([0, -10], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-25], 'k-', linewidth=4)
    ax1.plot([0, 10], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-25], 'k-', linewidth=4)
    ax1.plot([-25, 25], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN], 'g-', linewidth=5)
    ax1.fill_between([-25, 25], TARGET_HEIGHT_IN-3, TARGET_HEIGHT_IN+3, color='green', alpha=0.3)
    ax1.annotate('FUNNEL\n(opening UP)', (0, TARGET_HEIGHT_IN+12), ha='center', 
                fontsize=10, fontweight='bold')
    
    # Robot
    ax1.add_patch(Rectangle((distance_in-14, 0), 28, 20, color='gray', alpha=0.3))
    ax1.axhline(y=0, color='saddlebrown', linewidth=4)
    
    ax1.set_xlim(-40, distance_in + 40)
    ax1.set_ylim(-10, 260)
    ax1.set_xlabel('Distance from target (inches)', fontsize=12)
    ax1.set_ylabel('Height (inches)', fontsize=12)
    ax1.set_title(f'LOB Trajectory: {distance_ft} ft shot\nBall arcs UP then DESCENDS into funnel', fontsize=12)
    ax1.legend(loc='upper right', fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Right plot: Metrics
    ax2.axis('off')
    
    if solution:
        # Determine quality
        if solution['entry_angle_deg'] >= 60:
            quality = "IDEAL"
            quality_color = "green"
        elif solution['entry_angle_deg'] >= 45:
            quality = "GOOD"
            quality_color = "orange"
        else:
            quality = "RISKY"
            quality_color = "red"
        
        metrics_text = f"""
    LOB SHOT ANALYSIS
    {'='*45}
    
    INPUT PARAMETERS
    Distance:        {distance_ft} ft ({distance_in} in)
    Muzzle velocity: {velocity_fps} ft/s
    Robot velocity:  {robot_velocity_fps:+d} ft/s toward target
    Release height:  {RELEASE_HEIGHT_IN} in
    
    {'='*45}
    
    LOB TRAJECTORY RESULTS
    Launch angle:    {solution['angle_deg']:.1f} deg (HIGH lob)
    Entry angle:     {solution['entry_angle_deg']:.1f} deg (DESCENDING)
    Max height:      {solution['max_height']:.0f} in (above target!)
    Flight time:     {solution['t_flight']:.2f} s
    
    {'='*45}
    
    The ball goes UP {solution['max_height'] - TARGET_HEIGHT_IN:.0f}" above
    the target, then falls DOWN into the funnel.
    
    Entry quality: {quality}
        """
        
        ax2.text(0.05, 0.95, metrics_text, transform=ax2.transAxes, fontsize=11,
                verticalalignment='top', fontfamily='monospace',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
        
        ax2.text(0.5, 0.08, quality, transform=ax2.transAxes, fontsize=36,
                ha='center', color=quality_color, fontweight='bold')
    else:
        ax2.text(0.5, 0.5, 'No valid lob solution\nat this distance/velocity', 
                transform=ax2.transAxes, fontsize=16, ha='center', color='red')
    
    plt.tight_layout()
    plt.show()

# Create interactive widget
interact(plot_lob_trajectory_interactive,
         distance_ft=IntSlider(min=2, max=21, step=1, value=7, description='Distance (ft)'),
         velocity_fps=IntSlider(min=25, max=40, step=1, value=30, description='Velocity (fps)'),
         robot_velocity_fps=IntSlider(min=-10, max=15, step=1, value=0, description='Robot vel'));

---
## 4. Full Range Analysis (24" to 250")

Verify that 30 ft/s covers the entire alliance zone shooting range.

In [ ]:
def analyze_full_range(velocity_fps=RECOMMENDED_VELOCITY_FPS):
    """Analyze lob solutions across the full shooting range."""
    
    print(f"LOB SHOOTER ANALYSIS - {velocity_fps} ft/s")
    print("=" * 75)
    print(f"{'Distance':>10} | {'Launch':>8} | {'Entry':>8} | {'Max Ht':>8} | {'Flight':>8} | {'Quality':>10}")
    print("-" * 75)
    
    results = []
    distances_in = list(range(24, 260, 12))  # 24" to 252" in 12" steps
    
    for dist_in in distances_in:
        solution = find_lob_solution(dist_in, velocity_fps)
        
        if solution:
            # Determine quality
            if solution['entry_angle_deg'] >= 60:
                quality = "IDEAL"
            elif solution['entry_angle_deg'] >= 45:
                quality = "GOOD"
            else:
                quality = "RISKY"
            
            results.append({
                'distance_in': dist_in,
                'distance_ft': dist_in / 12,
                **solution,
                'quality': quality
            })
            
            print(f"{dist_in/12:>8.1f} ft | {solution['angle_deg']:>6.1f} deg | "
                  f"{solution['entry_angle_deg']:>6.1f} deg | {solution['max_height']:>6.0f}\" | "
                  f"{solution['t_flight']:>6.2f} s | {quality:>10}")
        else:
            print(f"{dist_in/12:>8.1f} ft | {'--- NO SOLUTION ---':^50}")
    
    print("=" * 75)
    
    # Summary
    if results:
        min_entry = min(r['entry_angle_deg'] for r in results)
        max_entry = max(r['entry_angle_deg'] for r in results)
        ideal_count = sum(1 for r in results if r['quality'] == 'IDEAL')
        good_count = sum(1 for r in results if r['quality'] == 'GOOD')
        
        print(f"\nSUMMARY:")
        print(f"  Coverage: {results[0]['distance_ft']:.0f} ft to {results[-1]['distance_ft']:.0f} ft")
        print(f"  Entry angles: {min_entry:.0f} to {max_entry:.0f} deg")
        print(f"  Quality: {ideal_count} IDEAL, {good_count} GOOD out of {len(results)} distances")
    
    return results

# Run the analysis
results = analyze_full_range(30)

---
## 5. Trajectory Sweep Visualization

See how the lob trajectories look across the full range.

In [ ]:
def plot_trajectory_sweep(velocity_fps=RECOMMENDED_VELOCITY_FPS):
    """Plot lob trajectories across the shooting range."""
    
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # Left: Trajectories
    ax1 = axes[0]
    distances_ft = [2, 5, 8, 12, 16, 21]
    colors = plt.cm.viridis(np.linspace(0.15, 0.9, len(distances_ft)))
    
    for dist_ft, color in zip(distances_ft, colors):
        dist_in = dist_ft * 12
        solution = find_lob_solution(dist_in, velocity_fps)
        
        if solution:
            t, x, y = calculate_trajectory(dist_in, velocity_fps, solution['angle_deg'])
            ax1.plot(x, y, color=color, linewidth=2.5,
                    label=f"{dist_ft} ft: {solution['angle_deg']:.0f} deg launch, "
                          f"{solution['entry_angle_deg']:.0f} deg entry")
    
    # Funnel target
    ax1.plot([0, -10], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-25], 'k-', linewidth=4)
    ax1.plot([0, 10], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-25], 'k-', linewidth=4)
    ax1.plot([-30, 30], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN], 'g-', linewidth=5)
    ax1.fill_between([-30, 30], TARGET_HEIGHT_IN-3, TARGET_HEIGHT_IN+3, color='green', alpha=0.3)
    ax1.annotate('FUNNEL (opening UP)', (0, TARGET_HEIGHT_IN+12), ha='center', fontsize=10, fontweight='bold')
    
    ax1.axhline(y=0, color='saddlebrown', linewidth=4)
    
    ax1.set_xlim(-50, 280)
    ax1.set_ylim(-10, 260)
    ax1.set_xlabel('Distance from target (inches)', fontsize=12)
    ax1.set_ylabel('Height (inches)', fontsize=12)
    ax1.set_title(f'LOB Trajectory Sweep @ {velocity_fps} ft/s\nAll balls DESCEND into funnel', fontsize=12)
    ax1.legend(loc='upper right', fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Right: Entry angle vs distance
    ax2 = axes[1]
    
    distances_range = np.linspace(24, 250, 50)
    entry_angles = []
    launch_angles = []
    
    for dist_in in distances_range:
        solution = find_lob_solution(dist_in, velocity_fps)
        if solution:
            entry_angles.append(solution['entry_angle_deg'])
            launch_angles.append(solution['angle_deg'])
        else:
            entry_angles.append(np.nan)
            launch_angles.append(np.nan)
    
    # Quality zones
    ax2.axhspan(60, 90, color='green', alpha=0.2, label='IDEAL (60-90 deg)')
    ax2.axhspan(45, 60, color='yellow', alpha=0.2, label='GOOD (45-60 deg)')
    ax2.axhspan(0, 45, color='red', alpha=0.15, label='RISKY (<45 deg)')
    
    ax2.plot(distances_range/12, entry_angles, 'b-', linewidth=3, label='Entry angle (descent)')
    ax2.plot(distances_range/12, launch_angles, 'r--', linewidth=2, label='Launch angle')
    
    ax2.set_xlim(1, 22)
    ax2.set_ylim(0, 95)
    ax2.set_xlabel('Distance (ft)', fontsize=12)
    ax2.set_ylabel('Angle (degrees)', fontsize=12)
    ax2.set_title(f'Entry Angle vs Distance @ {velocity_fps} ft/s\nSteeper entry = Better funnel capture', fontsize=12)
    ax2.legend(loc='right', fontsize=9)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Interactive version
interact(plot_trajectory_sweep,
         velocity_fps=IntSlider(min=30, max=40, step=1, value=30   , description='Velocity (fps)'));

In [ ]:
# Static version for export
def export_trajectory_plot():
    """Generate static trajectory plot for printing."""
    velocity_fps = 30
    
    fig, ax = plt.subplots(figsize=(16, 10))
    
    distances_ft = [2, 4, 6, 8, 10, 14, 18, 21]
    colors = plt.cm.viridis(np.linspace(0.1, 0.95, len(distances_ft)))
    
    for dist_ft, color in zip(distances_ft, colors):
        dist_in = dist_ft * 12
        solution = find_lob_solution(dist_in, velocity_fps)
        
        if solution:
            t, x, y = calculate_trajectory(dist_in, velocity_fps, solution['angle_deg'])
            ax.plot(x, y, color=color, linewidth=2.5,
                   label=f"{dist_ft} ft: {solution['angle_deg']:.0f} deg launch -> "
                         f"{solution['entry_angle_deg']:.0f} deg entry")
    
    # Funnel
    ax.plot([0, -12], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-30], 'k-', linewidth=5)
    ax.plot([0, 12], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-30], 'k-', linewidth=5)
    ax.plot([-35, 35], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN], 'g-', linewidth=6)
    ax.fill_between([-35, 35], TARGET_HEIGHT_IN-4, TARGET_HEIGHT_IN+4, color='green', alpha=0.3)
    ax.annotate('FUNNEL TARGET\n(opening faces UP)', (0, TARGET_HEIGHT_IN+15), 
               ha='center', fontsize=12, fontweight='bold')
    
    ax.axhline(y=0, color='saddlebrown', linewidth=5)
    
    ax.set_xlim(-60, 280)
    ax.set_ylim(-10, 260)
    ax.set_xlabel('Distance from target (inches)', fontsize=14)
    ax.set_ylabel('Height (inches)', fontsize=14)
    ax.set_title(f'LOB SHOOTER: Full Range Coverage @ {velocity_fps} ft/s\n'
                f'All trajectories arc UP then DESCEND into funnel', fontsize=14)
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

fig = export_trajectory_plot()
plt.show()

---
## 6. Robot Motion Effects

How does driving while shooting affect the lob trajectory?

In [ ]:
def plot_robot_velocity_effect(distance_ft=10, velocity_fps=34):
    """Show how robot motion affects the lob trajectory."""
    distance_in = distance_ft * 12
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))
    
    # Left: Trajectories at different robot velocities
    ax1 = axes[0]
    robot_vels = [-10, -5, 0, 5, 10]
    colors = plt.cm.coolwarm(np.linspace(0, 1, len(robot_vels)))
    
    for v_robot_fps, color in zip(robot_vels, colors):
        vx_robot = v_robot_fps * 12
        solution = find_lob_solution(distance_in, velocity_fps, vx_robot=vx_robot)
        
        if solution:
            t, x, y = calculate_trajectory(distance_in, velocity_fps, solution['angle_deg'],
                                          vx_robot=vx_robot)
            direction = "toward" if v_robot_fps > 0 else "away" if v_robot_fps < 0 else "stationary"
            ax1.plot(x, y, color=color, linewidth=2.5,
                    label=f'{abs(v_robot_fps)} fps {direction}: {solution["angle_deg"]:.0f} deg')
    
    # Funnel
    ax1.plot([0, -8], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-20], 'k-', linewidth=4)
    ax1.plot([0, 8], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-20], 'k-', linewidth=4)
    ax1.plot([-25, 25], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN], 'g-', linewidth=5)
    ax1.axhline(y=0, color='saddlebrown', linewidth=4)
    
    ax1.set_xlim(-30, distance_in + 30)
    ax1.set_ylim(-10, 240)
    ax1.set_xlabel('Distance (inches)', fontsize=12)
    ax1.set_ylabel('Height (inches)', fontsize=12)
    ax1.set_title(f'Robot Motion Effect @ {distance_ft} ft\nAll are valid lob shots!', fontsize=12)
    ax1.legend(loc='upper right', fontsize=9)
    ax1.grid(True, alpha=0.3)
    
    # Right: Angle compensation chart
    ax2 = axes[1]
    
    robot_vels_range = np.linspace(-15, 15, 50)
    angles = []
    entry_angles = []
    
    for v_robot_fps in robot_vels_range:
        vx_robot = v_robot_fps * 12
        solution = find_lob_solution(distance_in, velocity_fps, vx_robot=vx_robot)
        if solution:
            angles.append(solution['angle_deg'])
            entry_angles.append(solution['entry_angle_deg'])
        else:
            angles.append(np.nan)
            entry_angles.append(np.nan)
    
    ax2.plot(robot_vels_range, angles, 'b-', linewidth=3, label='Launch angle')
    ax2.plot(robot_vels_range, entry_angles, 'g--', linewidth=2, label='Entry angle')
    ax2.axvline(x=0, color='gray', linestyle=':', alpha=0.5)
    
    ax2.annotate('Driving TOWARD target\n= adjust angle slightly', 
                xy=(8, 75), fontsize=10, ha='center')
    ax2.annotate('Driving AWAY\n= adjust angle slightly',
                xy=(-8, 75), fontsize=10, ha='center')
    
    ax2.set_xlim(-15, 15)
    ax2.set_ylim(50, 95)
    ax2.set_xlabel('Robot velocity toward target (ft/s)', fontsize=12)
    ax2.set_ylabel('Angle (degrees)', fontsize=12)
    ax2.set_title('Angle Compensation for Robot Motion\n(Both stay in IDEAL zone)', fontsize=12)
    ax2.legend(loc='lower right', fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

interact(plot_robot_velocity_effect,
         distance_ft=IntSlider(min=4, max=18, step=2, value=10, description='Distance (ft)'),
         velocity_fps=IntSlider(min=0, max=38, step=1, value=30, description='Velocity (fps)'));

---
## 7. Flywheel Requirements for Lob Shooter

At 30 ft/s, we need LESS flywheel energy than a high-velocity shooter. This is a WIN!

In [ ]:
def calculate_flywheel_requirements(v_fps=45, num_balls=3, max_rpm_drop_pct=5, wheel_radius_in=2):
    """
    Calculate flywheel inertia requirements for multi-ball firing.
    
    Physics:
    - Energy stored in flywheel: E = ½Iω²
    - Energy transferred to ball: E = ½mv²
    - For small RPM drop: E_transferred << E_stored
    """
    # Convert units
    v_shot_m_s = v_fps * 12 * 0.0254  # m/s
    wheel_radius_m = wheel_radius_in * 0.0254  # m
    
    # Energy per ball
    E_ball = 0.5 * FUEL_MASS_KG * v_shot_m_s**2
    E_total = num_balls * E_ball
    
    # Angular velocity (assuming surface speed = ball speed)
    omega = v_shot_m_s / wheel_radius_m  # rad/s
    rpm = omega * 60 / (2 * np.pi)
    
    # For RPM drop of X%:
    # E_final/E_initial = (1 - X/100)²
    # E_transferred/E_initial = 1 - (1 - X/100)²
    drop_fraction = max_rpm_drop_pct / 100
    energy_fraction = 1 - (1 - drop_fraction)**2
    
    # Required stored energy
    E_required = E_total / energy_fraction
    
    # Required moment of inertia (total)
    I_total = 2 * E_required / omega**2
    I_per_wheel = I_total / 2  # Assuming 2 wheels
    
    # Convert to lb·in²
    I_per_wheel_lb_in2 = I_per_wheel / (0.0254**2 * 0.4536)
    
    # Estimate wheel mass (solid cylinder: I = ½mr²)
    wheel_mass_kg = 2 * I_per_wheel / wheel_radius_m**2
    wheel_mass_lb = wheel_mass_kg / 0.4536
    
    # Spin-up time estimate (2 NEO motors, ~2.6 N·m average)
    motor_torque = 2 * 2.6  # N·m
    spinup_time = I_total * omega / motor_torque
    
    return {
        'rpm': rpm,
        'energy_per_ball_J': E_ball,
        'total_energy_J': E_total,
        'stored_energy_J': E_required,
        'I_total_kg_m2': I_total,
        'I_per_wheel_lb_in2': I_per_wheel_lb_in2,
        'wheel_mass_lb': wheel_mass_lb,
        'spinup_time_s': spinup_time
    }

# Interactive display
def show_flywheel_analysis(v_fps=45, num_balls=3, max_drop=5, wheel_radius=2):
    results = calculate_flywheel_requirements(v_fps, num_balls, max_drop, wheel_radius)
    
    print("="*60)
    print("FLYWHEEL REQUIREMENTS")
    print("="*60)
    print(f"\nInputs:")
    print(f"  Muzzle velocity: {v_fps} ft/s")
    print(f"  Balls per burst: {num_balls}")
    print(f"  Max RPM drop: {max_drop}%")
    print(f"  Wheel radius: {wheel_radius}\"")
    print(f"\nOperating Point:")
    print(f"  Flywheel RPM: {results['rpm']:.0f}")
    print(f"\nEnergy Analysis:")
    print(f"  Energy per ball: {results['energy_per_ball_J']:.2f} J")
    print(f"  Total for {num_balls} balls: {results['total_energy_J']:.2f} J")
    print(f"  Required stored: {results['stored_energy_J']:.1f} J")
    print(f"\nInertia Requirements:")
    print(f"  Total I: {results['I_total_kg_m2']*1000:.2f} g·m²")
    print(f"  Per wheel: {results['I_per_wheel_lb_in2']:.2f} lb·in²")
    print(f"\nDesign Estimate (solid disk):")
    print(f"  Wheel mass needed: {results['wheel_mass_lb']:.2f} lb each")
    print(f"  Spin-up time (2 NEOs): {results['spinup_time_s']:.2f} s")
    print("="*60)

interact(show_flywheel_analysis,
         v_fps=IntSlider(min=30, max=60, step=5, value=45, description='Velocity (fps)'),
         num_balls=IntSlider(min=1, max=5, step=1, value=3, description='Balls/burst'),
         max_drop=FloatSlider(min=2, max=10, step=0.5, value=5, description='Max drop (%)'),
         wheel_radius=FloatSlider(min=1.5, max=4, step=0.5, value=2, description='Radius (in)'));

In [ ]:
# Compare different wheel sizes
print("WHEEL SIZE COMPARISON")
print("Target: 45 ft/s, 3 balls, <5% RPM drop")
print("="*70)
print(f"{'Diameter':>10} | {'RPM':>8} | {'Mass/wheel':>12} | {'Spin-up':>10} | {'Notes':>15}")
print("-"*70)

for diameter in [4, 6, 8, 10, 12]:
    radius = diameter / 2
    r = calculate_flywheel_requirements(45, 3, 5, radius)
    
    # Assessment
    if r['rpm'] > 4000:
        notes = "High RPM!"
    elif r['rpm'] < 1000:
        notes = "Very low RPM"
    elif r['spinup_time_s'] > 2:
        notes = "Slow recovery"
    else:
        notes = "Good balance"
    
    print(f"{diameter:>8}\" | {r['rpm']:>8.0f} | {r['wheel_mass_lb']:>10.2f} lb | "
          f"{r['spinup_time_s']:>8.2f} s | {notes:>15}")

print("="*70)

---
## 8. R106 Extension Analysis

Analyzing the extension envelope for a 180° pivoting shooter.

In [ ]:
def plot_extension_envelope(barrel_length=10, pivot_height=15):
    """
    Visualize the barrel sweep envelope for R106 compliance.
    """
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Robot frame (28" x 20" assumed)
    frame_width = 28
    frame_height = 20
    ax.add_patch(Rectangle((-frame_width/2, 0), frame_width, frame_height,
                           color='gray', alpha=0.3, label='Robot frame'))
    
    # 12" extension limits
    limit = 12
    ax.axvline(x=frame_width/2 + limit, color='red', linestyle='--', linewidth=2, 
               label='12" extension limit')
    ax.axvline(x=-frame_width/2 - limit, color='red', linestyle='--', linewidth=2)
    ax.axhline(y=ROBOT_MAX_HEIGHT_IN, color='red', linestyle=':', linewidth=2,
               label='30" height limit')
    
    # Pivot point
    ax.plot(0, pivot_height, 'ko', markersize=12, label='Pivot point')
    
    # Draw barrel at multiple angles
    angles = np.linspace(0, np.pi, 37)  # 180° sweep, every 5°
    tip_x = []
    tip_y = []
    violations = []
    
    for theta in angles:
        # Barrel tip position (theta=0 is straight down, theta=π/2 is horizontal right)
        bx = barrel_length * np.cos(theta - np.pi/2)
        by = pivot_height + barrel_length * np.sin(theta - np.pi/2)
        tip_x.append(bx)
        tip_y.append(by)
        
        # Check for extension violations
        extends_right = bx > frame_width/2
        extends_left = bx < -frame_width/2
        extends_up = by > frame_height
        
        num_directions = extends_right + extends_left + extends_up
        
        # Draw barrel
        alpha = 0.15 if np.degrees(theta) % 15 != 0 else 0.6
        ax.plot([0, bx], [pivot_height, by], 'b-', linewidth=2, alpha=alpha)
        
        if num_directions > 1:
            violations.append((bx, by))
    
    # Draw envelope
    ax.plot(tip_x, tip_y, 'g-', linewidth=3, label='Barrel sweep')
    
    # Mark violations
    for vx, vy in violations:
        ax.plot(vx, vy, 'r*', markersize=15)
    
    if violations:
        ax.plot([], [], 'r*', markersize=15, label=f'Multi-direction ext ({len(violations)} pts)')
    
    ax.set_xlim(-35, 35)
    ax.set_ylim(-5, 40)
    ax.set_xlabel('X Position (inches)', fontsize=12)
    ax.set_ylabel('Y Position (inches)', fontsize=12)
    ax.set_title(f'R106 Extension Envelope Analysis\nBarrel: {barrel_length}", Pivot height: {pivot_height}"', fontsize=12)
    ax.legend(loc='upper left', fontsize=10)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    # Compliance assessment
    if len(violations) == 0:
        status = "R106 COMPLIANT"
        color = 'green'
    else:
        status = f"POTENTIAL VIOLATIONS ({len(violations)} positions)"
        color = 'red'
    
    ax.text(0, -3, status, ha='center', fontsize=14, fontweight='bold', color=color)
    
    plt.tight_layout()
    plt.show()

interact(plot_extension_envelope,
         barrel_length=IntSlider(min=6, max=16, step=1, value=10, description='Barrel (in)'),
         pivot_height=IntSlider(min=10, max=20, step=1, value=15, description='Pivot ht (in)'));

---
## 9. Driver Aiming Lookup Table

Generate a lookup table for driver practice.

In [ ]:
def generate_lookup_table(y0=18, v_fps=30):
    """
    Generate and display a driver aiming lookup table for LOB shots.
    """
    distances_ft = list(range(3, 19))
    robot_vels = [-10, -5, 0, 5, 10]
    
    # Create table data
    table = np.zeros((len(distances_ft), len(robot_vels)))
    
    for i, d_ft in enumerate(distances_ft):
        for j, v_robot_fps in enumerate(robot_vels):
            x0 = d_ft * 12
            vx_robot = v_robot_fps * 12
            solution = find_lob_solution(x0, v_fps, y0, TARGET_HEIGHT_IN, vx_robot)
            if solution:
                table[i, j] = solution['angle_deg']
            else:
                table[i, j] = np.nan
    
    # Display as heatmap
    fig, ax = plt.subplots(figsize=(12, 10))
    
    im = ax.imshow(table, aspect='auto', cmap='viridis',
                   extent=[robot_vels[0]-2.5, robot_vels[-1]+2.5,
                          distances_ft[-1]+0.5, distances_ft[0]-0.5])
    
    # Add text annotations
    for i, d_ft in enumerate(distances_ft):
        for j, v_robot in enumerate(robot_vels):
            val = table[i, j]
            if not np.isnan(val):
                ax.text(v_robot, d_ft, f'{val:.0f}', ha='center', va='center',
                       fontsize=9, color='white', fontweight='bold')
    
    ax.set_xlabel('Robot Velocity Toward Target (ft/s)', fontsize=12)
    ax.set_ylabel('Distance from Target (ft)', fontsize=12)
    ax.set_title(f'LOB SHOOTER AIMING TABLE
Release: {y0}", Muzzle: {v_fps} ft/s', fontsize=14)
    
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Launch Angle (degrees)', fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    # Print text version
    print("
TEXT VERSION (for pit display):")
    print("="*65)
    header = f"{'Dist':>6} |" + "".join([f" {v:>+5}fps |" for v in robot_vels])
    print(header)
    print("-"*65)
    for i, d_ft in enumerate(distances_ft):
        row = f"{d_ft:>4} ft |"
        for j in range(len(robot_vels)):
            val = table[i, j]
            if np.isnan(val):
                row += "   --- |"
            else:
                row += f" {val:>5.0f}  |"
        print(row)
    print("="*65)

interact(generate_lookup_table,
         y0=IntSlider(min=14, max=22, step=2, value=18, description='Release (in)'),
         v_fps=IntSlider(min=28, max=40, step=2, value=30, description='Velocity (fps)'));

---
## 10. Summary: Design Recommendations

Based on the analysis above, here are the key takeaways:

In [ ]:
summary = """
╔══════════════════════════════════════════════════════════════════════════╗
║                    SHOOTER DESIGN RECOMMENDATIONS                        ║
╚══════════════════════════════════════════════════════════════════════════╝

1. OPTIMAL SHOOTING DISTANCE
   ──────────────────────────
   • 4-6 feet: Best entry angles (35-50°) for funnel target
   • 7-8 feet: Marginal - may bounce at high volumes
   • 9+ feet: Risky - shallow entry angles (<25°)
   
   RECOMMENDATION: Design strategy around 5-7 ft shots

2. MUZZLE VELOCITY
   ────────────────
   • 40-50 ft/s provides good balance
   • Higher velocity = slightly better entry at long range
   • But higher velocity = more energy needed in flywheel
   
   RECOMMENDATION: Target 45 ft/s muzzle velocity

3. FLYWHEEL SIZING (for 3-ball burst, <5% drop)
   ─────────────────────────────────────────────
   • 4" diameter wheels: 1,289 RPM, 14.5 lb each, 1.76s spin-up
   • 6" diameter wheels: 859 RPM, 14.5 lb each, 2.64s spin-up
   
   RECOMMENDATION: 4" wheels with steel rims for inertia

4. SHOOTING WHILE MOVING
   ──────────────────────
   • Driving toward target increases required angle by ~5°/10fps
   • Benefits: faster cycles, maintains flywheel momentum
   • Use lookup table for consistent angle compensation
   
   RECOMMENDATION: Practice shooting at constant robot speed

5. R106 COMPLIANCE (180° PIVOT)
   ─────────────────────────────
   • Keep barrel length ≤ 10" to stay within frame
   • Place pivot point near top of robot
   • Use slip rings for power/signal through rotation
   • Implement software interlocks for illegal positions
   
   RECOMMENDATION: 10" barrel, centered pivot at 15" height

══════════════════════════════════════════════════════════════════════════
"""

print(summary)

---
## 11. Export Data for Practice

Generate files for driver practice and pit display.

In [ ]:
# Generate all plots for printing
def export_all_plots():
    import os
    
    # 1. Distance sweep
    plot_distance_sweep(18, 45, 0)
    plt.savefig('export_distance_sweep.png', dpi=150, bbox_inches='tight')
    print('Saved: export_distance_sweep.png')
    
    # 2. Robot velocity effect
    plot_robot_velocity_effect(7, 18, 45)
    plt.savefig('export_velocity_effect.png', dpi=150, bbox_inches='tight')
    print('Saved: export_velocity_effect.png')
    
    # 3. Extension envelope
    plot_extension_envelope(10, 15)
    plt.savefig('export_extension.png', dpi=150, bbox_inches='tight')
    print('Saved: export_extension.png')
    
    # 4. Lookup table
    generate_lookup_table(18, 45)
    plt.savefig('export_lookup_table.png', dpi=150, bbox_inches='tight')
    print('Saved: export_lookup_table.png')
    
    print('\nAll exports complete!')

# Uncomment to generate exports:
# export_all_plots()

---
## Questions for Students

Use these discussion questions during your design reviews:

1. **Why does the entry angle decrease with distance?** What physical principle explains this?

2. **If we increase muzzle velocity, why doesn't the entry angle improve significantly?** 

3. **Why do we need MORE flywheel inertia for smaller diameter wheels?**

4. **What happens to our aiming if the robot hits a bump while shooting?** How could we compensate?

5. **Why might we choose to shoot while driving AWAY from the target?** When would this be useful?

6. **What's the tradeoff between barrel length and R106 compliance?**

---
## 12. Integrated Shooter System Analysis

This section combines trajectory analysis with the shooter drivetrain model:
- Wide roller shooter (N game pieces wide)
- Closed-loop motor control (maintains target RPM)
- Recovery time is the key metric
- Rate of fire calculations

---
## 12. Integrated Shooter System Analysis

This section combines trajectory analysis with the shooter drivetrain model:
- Wide roller shooter (N game pieces wide)
- Closed-loop motor control (maintains target RPM)
- Recovery time is the key metric
- Rate of fire calculations

In [ ]:
# Import the shooter system module
from shooter_system import (
    WideRollerShooter, IntegratedShooterAnalysis, MOTORS,
    find_lob_solution, calculate_trajectory_points,
    plot_shooter_performance, plot_integrated_analysis,
    compare_configurations, calculate_optimal_gear_ratio,
    TARGET_HEIGHT_IN, FUEL_DIAMETER_IN, FUEL_MASS_KG
)

print("Shooter System Module Loaded")
print("="*50)
print("Available motors:", list(MOTORS.keys()))
print(f"Target height: {TARGET_HEIGHT_IN}\"")
print(f"Ball diameter: {FUEL_DIAMETER_IN}\"")
print("="*50)

### 12.1 Interactive Wide Roller Shooter Configuration

Configure your shooter and see real-time performance analysis.

**Key parameters:**
- **Roller width** = How many balls the roller can shoot simultaneously
- **Gear ratio** = Motor:Roller reduction (higher = more torque, lower max speed)
- **Added flywheel** = Extra inertia to reduce RPM drop per burst

In [ ]:
def analyze_shooter_interactive(motor_type='KRAKENX60', num_motors=2, gear_ratio=1.5,
                                  roller_diameter=4.0, roller_width=1, 
                                  added_flywheel=0, target_velocity=30,
                                  release_height=18):
    """Interactive shooter analysis with trajectory integration."""
    
    # Create shooter
    shooter = WideRollerShooter(
        motor_type=motor_type,
        num_motors=num_motors,
        gear_ratio=gear_ratio,
        roller_diameter_in=roller_diameter,
        roller_width_game_pieces=roller_width,
        added_flywheel_inertia_lb_in2=added_flywheel,
        energy_transfer_efficiency=0.80,
    )
    
    # Run analysis
    result = shooter.simulate_burst_cycle(target_velocity, roller_width)
    
    # Create figure
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    if not result['feasible']:
        fig.text(0.5, 0.5, f"NOT FEASIBLE\n{result['reason']}", 
                ha='center', va='center', fontsize=20, color='red')
        plt.tight_layout()
        plt.show()
        return
    
    # Plot 1: Trajectories with shooter velocity
    ax1 = axes[0, 0]
    ball_v = result['exit_velocity_fps']
    distances_ft = [3, 5, 7, 10, 12, 15, 18]
    colors = plt.cm.viridis(np.linspace(0.15, 0.9, len(distances_ft)))
    
    for d_ft, color in zip(distances_ft, colors):
        d_in = d_ft * 12
        traj = find_lob_solution(d_in, ball_v, release_height)
        if traj:
            x, y = calculate_trajectory_points(d_in, traj['angle_deg'], ball_v, release_height)
            ax1.plot(x, y, color=color, linewidth=2,
                    label=f"{d_ft}ft: {traj['angle_deg']:.0f}° → {traj['entry_angle_deg']:.0f}°")
    
    # Funnel target
    ax1.plot([0, -10], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-25], 'k-', linewidth=3)
    ax1.plot([0, 10], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN-25], 'k-', linewidth=3)
    ax1.plot([-25, 25], [TARGET_HEIGHT_IN, TARGET_HEIGHT_IN], 'g-', linewidth=4)
    ax1.axhline(y=0, color='saddlebrown', linewidth=3)
    ax1.axhline(y=release_height, color='blue', linestyle=':', alpha=0.5, label=f'Release: {release_height}"')
    
    ax1.set_xlim(-40, 230)
    ax1.set_ylim(-10, 220)
    ax1.set_xlabel('Distance from target (inches)')
    ax1.set_ylabel('Height (inches)')
    ax1.set_title(f'Lob Trajectories @ {ball_v:.1f} ft/s')
    ax1.legend(loc='upper right', fontsize=8)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Recovery curve
    ax2 = axes[0, 1]
    target_rpm = result['target_rpm']
    rpm_after = result['rpm_after_burst']
    
    times = []
    rpms = []
    rpm = rpm_after
    t = 0
    dt = 0.0005
    
    while rpm < target_rpm * 0.99 and t < 0.2:
        times.append(t * 1000)
        rpms.append(rpm)
        omega = rpm * (2 * np.pi / 60)
        torque = shooter.torque_at_roller(rpm)
        alpha = torque / shooter.total_inertia_kg_m2
        omega += alpha * dt
        rpm = min(omega * 60 / (2 * np.pi), shooter.max_roller_rpm())
        t += dt
    
    ax2.plot(times, rpms, 'b-', linewidth=2.5)
    ax2.axhline(y=target_rpm, color='g', linestyle='--', linewidth=2, label=f'Target: {target_rpm:.0f} RPM')
    ax2.axhline(y=rpm_after, color='r', linestyle=':', linewidth=2, label=f'After burst: {rpm_after:.0f} RPM')
    ax2.fill_between([0, result['recovery_time_s']*1000], [rpm_after, target_rpm], 
                     color='lightblue', alpha=0.3)
    ax2.set_xlabel('Time (ms)')
    ax2.set_ylabel('Roller RPM')
    ax2.set_title(f'Recovery After {roller_width}-Ball Burst')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Rate vs roller width
    ax3 = axes[1, 0]
    widths = [1, 2, 3, 4, 5, 6]
    rates = []
    recovery_times = []
    
    for w in widths:
        test_shooter = WideRollerShooter(
            motor_type=motor_type, num_motors=num_motors, gear_ratio=gear_ratio,
            roller_diameter_in=roller_diameter, roller_width_game_pieces=w,
            added_flywheel_inertia_lb_in2=added_flywheel
        )
        r = test_shooter.simulate_burst_cycle(target_velocity, w)
        if r['feasible']:
            rates.append(r['balls_per_second'])
            recovery_times.append(r['recovery_time_s'] * 1000)
        else:
            rates.append(0)
            recovery_times.append(0)
    
    ax3.bar([str(w) for w in widths], rates, color='steelblue', alpha=0.7)
    ax3.axhline(y=rates[roller_width-1] if roller_width <= len(rates) else 0, 
                color='red', linestyle='--', linewidth=2, label='Current config')
    ax3.set_xlabel('Roller Width (balls)')
    ax3.set_ylabel('Rate of Fire (balls/sec)')
    ax3.set_title('Rate of Fire vs Roller Width')
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Plot 4: Summary info
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    motor_name = MOTORS[motor_type].name
    
    summary = f"""
    SHOOTER SYSTEM CONFIGURATION
    {'='*50}
    
    Motor: {motor_name} × {num_motors}
    Gear ratio: {gear_ratio:.2f}:1 (motor:roller)
    Roller: {roller_diameter:.1f}" dia × {roller_width} balls wide
    Added flywheel: {added_flywheel:.0f} lb-in²
    Total inertia: {shooter.total_inertia_lb_in2:.2f} lb-in²
    
    {'='*50}
    
    PERFORMANCE @ {target_velocity} ft/s
    Required RPM: {target_rpm:.0f}
    Max RPM: {result['max_rpm']:.0f} ({result['headroom_pct']:.0f}% headroom)
    
    Spinup from rest: {result['spinup_time_s']*1000:.0f} ms
    RPM drop per burst: {result['rpm_drop_pct']:.1f}%
    Recovery time: {result['recovery_time_s']*1000:.0f} ms
    
    {'='*50}
    
    RATE OF FIRE
    {result['balls_per_second']:.1f} balls/second
    {result['bursts_per_second']:.1f} bursts/second
    """
    
    ax4.text(0.05, 0.95, summary, transform=ax4.transAxes, fontsize=11,
             verticalalignment='top', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))
    
    plt.suptitle(f'Integrated Shooter Analysis - {roller_width}-Wide @ {target_velocity} ft/s', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Create interactive widget
motor_dropdown = Dropdown(options=['NEO', 'FALCON500', 'KRAKENX60'], value='KRAKENX60', description='Motor')

interact(analyze_shooter_interactive,
         motor_type=motor_dropdown,
         num_motors=IntSlider(min=1, max=4, value=2, description='# Motors'),
         gear_ratio=FloatSlider(min=0.5, max=3.0, step=0.1, value=1.5, description='Gear Ratio'),
         roller_diameter=FloatSlider(min=3.0, max=6.0, step=0.5, value=4.0, description='Roller (in)'),
         roller_width=IntSlider(min=1, max=6, value=1, description='Width (balls)'),
         added_flywheel=FloatSlider(min=0, max=30, step=2, value=0, description='+Flywheel'),
         target_velocity=IntSlider(min=25, max=40, value=30, description='Target (fps)'),
         release_height=IntSlider(min=14, max=22, value=18, description='Release (in)'));

### 12.2 Motor & Configuration Comparison

Compare different motor choices and configurations side-by-side.

In [ ]:
def compare_shooters_interactive(target_velocity=30, roller_width=1):
    """Compare all motor configurations."""
    compare_configurations(target_velocity, roller_width)
    
    # Visual comparison
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    motors = ['NEO', 'FALCON500', 'KRAKENX60']
    colors = ['#2ecc71', '#3498db', '#e74c3c']
    
    spinup_times = []
    recovery_times = []
    rates = []
    
    for motor in motors:
        shooter = WideRollerShooter(
            motor_type=motor, num_motors=2, gear_ratio=1.5,
            roller_diameter_in=4.0, roller_width_game_pieces=roller_width
        )
        r = shooter.simulate_burst_cycle(target_velocity, roller_width)
        if r['feasible']:
            spinup_times.append(r['spinup_time_s'] * 1000)
            recovery_times.append(r['recovery_time_s'] * 1000)
            rates.append(r['balls_per_second'])
        else:
            spinup_times.append(0)
            recovery_times.append(0)
            rates.append(0)
    
    # Spinup
    axes[0].bar(motors, spinup_times, color=colors)
    axes[0].set_ylabel('Time (ms)')
    axes[0].set_title('Spinup Time')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # Recovery
    axes[1].bar(motors, recovery_times, color=colors)
    axes[1].set_ylabel('Time (ms)')
    axes[1].set_title('Recovery Time')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Rate
    axes[2].bar(motors, rates, color=colors)
    axes[2].set_ylabel('Balls/second')
    axes[2].set_title('Rate of Fire')
    axes[2].grid(True, alpha=0.3, axis='y')
    
    plt.suptitle(f'Motor Comparison: {roller_width}-Wide Roller @ {target_velocity} ft/s', 
                 fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()

interact(compare_shooters_interactive,
         target_velocity=IntSlider(min=25, max=40, value=30, description='Target (fps)'),
         roller_width=IntSlider(min=1, max=4, value=1, description='Width (balls)'));

### 12.3 Optimal Gear Ratio Calculator

Find the best gear ratio for your motor/roller combination to maximize rate of fire.

In [ ]:
def optimize_gear_ratio(motor_type='KRAKENX60', num_motors=2, roller_diameter=4.0, 
                        roller_width=1, target_velocity=30, headroom_target=20):
    """Find optimal gear ratio for maximum rate of fire."""
    
    gear_ratios = np.linspace(0.5, 4.0, 50)
    rates = []
    recovery_times = []
    headrooms = []
    feasible = []
    
    for gr in gear_ratios:
        shooter = WideRollerShooter(
            motor_type=motor_type, num_motors=num_motors, gear_ratio=gr,
            roller_diameter_in=roller_diameter, roller_width_game_pieces=roller_width
        )
        r = shooter.simulate_burst_cycle(target_velocity, roller_width)
        
        if r['feasible']:
            rates.append(r['balls_per_second'])
            recovery_times.append(r['recovery_time_s'] * 1000)
            headrooms.append(r['headroom_pct'])
            feasible.append(True)
        else:
            rates.append(0)
            recovery_times.append(np.nan)
            headrooms.append(0)
            feasible.append(False)
    
    # Find optimal
    valid_rates = [(gr, rate) for gr, rate, f in zip(gear_ratios, rates, feasible) if f]
    if valid_rates:
        optimal_gr, optimal_rate = max(valid_rates, key=lambda x: x[1])
    else:
        optimal_gr, optimal_rate = None, 0
    
    # Plot results
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # Rate of fire
    ax1 = axes[0]
    ax1.plot(gear_ratios, rates, 'b-', linewidth=2.5)
    if optimal_gr:
        ax1.axvline(x=optimal_gr, color='green', linestyle='--', linewidth=2, 
                    label=f'Optimal: {optimal_gr:.2f}')
        ax1.plot(optimal_gr, optimal_rate, 'go', markersize=12)
    ax1.set_xlabel('Gear Ratio (motor:roller)')
    ax1.set_ylabel('Rate of Fire (balls/s)')
    ax1.set_title('Rate of Fire vs Gear Ratio')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Recovery time
    ax2 = axes[1]
    ax2.plot(gear_ratios, recovery_times, 'r-', linewidth=2.5)
    if optimal_gr:
        ax2.axvline(x=optimal_gr, color='green', linestyle='--', linewidth=2)
    ax2.set_xlabel('Gear Ratio (motor:roller)')
    ax2.set_ylabel('Recovery Time (ms)')
    ax2.set_title('Recovery Time vs Gear Ratio')
    ax2.grid(True, alpha=0.3)
    
    # Headroom
    ax3 = axes[2]
    ax3.fill_between(gear_ratios, 0, headrooms, alpha=0.3, color='blue')
    ax3.plot(gear_ratios, headrooms, 'b-', linewidth=2.5)
    ax3.axhline(y=headroom_target, color='orange', linestyle='--', linewidth=2, 
                label=f'Target: {headroom_target}%')
    if optimal_gr:
        ax3.axvline(x=optimal_gr, color='green', linestyle='--', linewidth=2)
    ax3.set_xlabel('Gear Ratio (motor:roller)')
    ax3.set_ylabel('Headroom (%)')
    ax3.set_title('RPM Headroom vs Gear Ratio')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.suptitle(f'{motor_type} × {num_motors} | {roller_diameter}" roller × {roller_width} wide | {target_velocity} fps',
                 fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print recommendation
    if optimal_gr:
        print(f"\n{'='*60}")
        print(f"OPTIMAL GEAR RATIO: {optimal_gr:.2f}:1")
        print(f"{'='*60}")
        print(f"Rate of fire: {optimal_rate:.1f} balls/second")
        
        # Get full stats at optimal
        shooter = WideRollerShooter(
            motor_type=motor_type, num_motors=num_motors, gear_ratio=optimal_gr,
            roller_diameter_in=roller_diameter, roller_width_game_pieces=roller_width
        )
        r = shooter.simulate_burst_cycle(target_velocity, roller_width)
        print(f"Recovery time: {r['recovery_time_s']*1000:.0f} ms")
        print(f"RPM headroom: {r['headroom_pct']:.0f}%")
        print(f"{'='*60}")
    else:
        print("No feasible gear ratio found for these parameters!")

# Interactive widget
interact(optimize_gear_ratio,
         motor_type=Dropdown(options=['NEO', 'FALCON500', 'KRAKENX60'], value='KRAKENX60', description='Motor'),
         num_motors=IntSlider(min=1, max=4, value=2, description='# Motors'),
         roller_diameter=FloatSlider(min=3.0, max=6.0, step=0.5, value=4.0, description='Roller (in)'),
         roller_width=IntSlider(min=1, max=6, value=1, description='Width'),
         target_velocity=IntSlider(min=25, max=40, value=30, description='Target (fps)'),
         headroom_target=IntSlider(min=10, max=40, value=20, description='Headroom %'));

### 12.4 Flywheel Sizing Helper

Calculate how much added flywheel inertia you need to achieve a target RPM drop percentage.

In [ ]:
def flywheel_sizing_helper(motor_type='KRAKENX60', num_motors=2, gear_ratio=1.5,
                           roller_diameter=4.0, roller_width=2, target_velocity=30,
                           max_rpm_drop=5.0, max_recovery_ms=100):
    """Find required flywheel inertia for target performance."""
    
    # Test range of flywheel inertias
    flywheel_range = np.linspace(0, 50, 100)
    rpm_drops = []
    recovery_times = []
    rates = []
    
    for fw in flywheel_range:
        shooter = WideRollerShooter(
            motor_type=motor_type, num_motors=num_motors, gear_ratio=gear_ratio,
            roller_diameter_in=roller_diameter, roller_width_game_pieces=roller_width,
            added_flywheel_inertia_lb_in2=fw
        )
        r = shooter.simulate_burst_cycle(target_velocity, roller_width)
        
        if r['feasible']:
            rpm_drops.append(r['rpm_drop_pct'])
            recovery_times.append(r['recovery_time_s'] * 1000)
            rates.append(r['balls_per_second'])
        else:
            rpm_drops.append(np.nan)
            recovery_times.append(np.nan)
            rates.append(0)
    
    # Find minimum flywheel for RPM drop target
    min_fw_for_drop = None
    for fw, drop in zip(flywheel_range, rpm_drops):
        if not np.isnan(drop) and drop <= max_rpm_drop:
            min_fw_for_drop = fw
            break
    
    # Find minimum flywheel for recovery time target (looking for best rate at that recovery)
    best_fw_for_recovery = None
    best_rate = 0
    for fw, rec, rate in zip(flywheel_range, recovery_times, rates):
        if not np.isnan(rec) and rec <= max_recovery_ms and rate > best_rate:
            best_fw_for_recovery = fw
            best_rate = rate
    
    # Plot
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # RPM drop
    ax1 = axes[0]
    ax1.plot(flywheel_range, rpm_drops, 'b-', linewidth=2.5)
    ax1.axhline(y=max_rpm_drop, color='red', linestyle='--', linewidth=2, label=f'Target: {max_rpm_drop}%')
    if min_fw_for_drop is not None:
        ax1.axvline(x=min_fw_for_drop, color='green', linestyle='--', linewidth=2,
                    label=f'Min needed: {min_fw_for_drop:.1f} lb-in²')
        ax1.plot(min_fw_for_drop, max_rpm_drop, 'go', markersize=12)
    ax1.set_xlabel('Added Flywheel Inertia (lb-in²)')
    ax1.set_ylabel('RPM Drop (%)')
    ax1.set_title(f'RPM Drop per {roller_width}-Ball Burst')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Recovery time
    ax2 = axes[1]
    ax2.plot(flywheel_range, recovery_times, 'r-', linewidth=2.5)
    ax2.axhline(y=max_recovery_ms, color='orange', linestyle='--', linewidth=2, 
                label=f'Target: {max_recovery_ms} ms')
    ax2.set_xlabel('Added Flywheel Inertia (lb-in²)')
    ax2.set_ylabel('Recovery Time (ms)')
    ax2.set_title('Recovery Time vs Flywheel Inertia')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Rate of fire
    ax3 = axes[2]
    ax3.plot(flywheel_range, rates, 'g-', linewidth=2.5)
    if best_fw_for_recovery is not None:
        ax3.axvline(x=best_fw_for_recovery, color='purple', linestyle='--', linewidth=2,
                    label=f'Best for {max_recovery_ms}ms: {best_fw_for_recovery:.1f} lb-in²')
        ax3.plot(best_fw_for_recovery, best_rate, 'mo', markersize=12)
    ax3.set_xlabel('Added Flywheel Inertia (lb-in²)')
    ax3.set_ylabel('Rate of Fire (balls/s)')
    ax3.set_title('Rate of Fire vs Flywheel Inertia')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    plt.suptitle(f'Flywheel Sizing: {motor_type} × {num_motors} | {gear_ratio}:1 | {roller_width}-wide @ {target_velocity} fps',
                 fontsize=12, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Recommendation
    print(f"\n{'='*70}")
    print("FLYWHEEL SIZING RECOMMENDATIONS")
    print(f"{'='*70}")
    
    # Base shooter inertia
    base_shooter = WideRollerShooter(
        motor_type=motor_type, num_motors=num_motors, gear_ratio=gear_ratio,
        roller_diameter_in=roller_diameter, roller_width_game_pieces=roller_width,
        added_flywheel_inertia_lb_in2=0
    )
    print(f"Base roller inertia: {base_shooter.total_inertia_lb_in2:.2f} lb-in²")
    
    if min_fw_for_drop is not None:
        print(f"\nFor ≤{max_rpm_drop}% RPM drop:")
        print(f"  Add ≥{min_fw_for_drop:.1f} lb-in² flywheel inertia")
        print(f"  Total system inertia: {base_shooter.total_inertia_lb_in2 + min_fw_for_drop:.1f} lb-in²")
    else:
        print(f"\nCannot achieve {max_rpm_drop}% RPM drop with up to 50 lb-in² added")
    
    if best_fw_for_recovery is not None:
        print(f"\nFor ≤{max_recovery_ms}ms recovery (max rate):")
        print(f"  Use {best_fw_for_recovery:.1f} lb-in² added flywheel")
        print(f"  Rate of fire: {best_rate:.1f} balls/second")
    
    print(f"{'='*70}")

# Interactive widget
interact(flywheel_sizing_helper,
         motor_type=Dropdown(options=['NEO', 'FALCON500', 'KRAKENX60'], value='KRAKENX60', description='Motor'),
         num_motors=IntSlider(min=1, max=4, value=2, description='# Motors'),
         gear_ratio=FloatSlider(min=0.5, max=3.0, step=0.1, value=1.5, description='Gear Ratio'),
         roller_diameter=FloatSlider(min=3.0, max=6.0, step=0.5, value=4.0, description='Roller (in)'),
         roller_width=IntSlider(min=1, max=6, value=2, description='Width'),
         target_velocity=IntSlider(min=25, max=40, value=30, description='Target (fps)'),
         max_rpm_drop=FloatSlider(min=1, max=15, step=0.5, value=5.0, description='Max drop %'),
         max_recovery_ms=IntSlider(min=50, max=200, step=10, value=100, description='Max rec (ms)'));

### 12.5 Quick Design Summary

Run this cell to generate a complete design summary for your chosen configuration.

In [ ]:
def generate_design_summary(motor_type='KRAKENX60', num_motors=2, gear_ratio=1.5,
                            roller_diameter=4.0, roller_width=2, added_flywheel=0,
                            target_velocity=30, release_height=18):
    """Generate a complete design summary for the chosen configuration."""
    
    # Create shooter
    shooter = WideRollerShooter(
        motor_type=motor_type, num_motors=num_motors, gear_ratio=gear_ratio,
        roller_diameter_in=roller_diameter, roller_width_game_pieces=roller_width,
        added_flywheel_inertia_lb_in2=added_flywheel, energy_transfer_efficiency=0.80
    )
    
    # Simulate performance
    result = shooter.simulate_burst_cycle(target_velocity, roller_width)
    
    # Test trajectory coverage
    distances_ft = [3, 5, 7, 10, 12, 15, 18]
    trajectory_coverage = []
    for d_ft in distances_ft:
        sol = find_lob_solution(d_ft * 12, result['exit_velocity_fps'], release_height)
        if sol:
            trajectory_coverage.append((d_ft, sol['angle_deg'], sol['entry_angle_deg']))
    
    # Print comprehensive summary
    motor = MOTORS[motor_type]
    
    print("╔" + "═"*70 + "╗")
    print("║" + " SHOOTER DESIGN SUMMARY ".center(70) + "║")
    print("╠" + "═"*70 + "╣")
    
    print("║ MECHANICAL CONFIGURATION".ljust(70) + " ║")
    print("║" + "-"*70 + "║")
    print(f"║  Motor: {motor.name} × {num_motors}".ljust(70) + " ║")
    print(f"║  Gear ratio: {gear_ratio:.2f}:1 (motor:roller)".ljust(70) + " ║")
    print(f"║  Roller: {roller_diameter:.1f}\" diameter × {roller_width} balls wide".ljust(70) + " ║")
    print(f"║  Added flywheel: {added_flywheel:.1f} lb-in²".ljust(70) + " ║")
    print(f"║  Total inertia: {shooter.total_inertia_lb_in2:.2f} lb-in²".ljust(70) + " ║")
    
    print("║" + " "*70 + "║")
    print("║ PERFORMANCE @ {:.0f} ft/s".format(target_velocity).ljust(70) + " ║")
    print("║" + "-"*70 + "║")
    
    if result['feasible']:
        print(f"║  Required roller RPM: {result['target_rpm']:.0f}".ljust(70) + " ║")
        print(f"║  Max roller RPM: {result['max_rpm']:.0f} ({result['headroom_pct']:.0f}% headroom)".ljust(70) + " ║")
        print(f"║  Actual exit velocity: {result['exit_velocity_fps']:.1f} ft/s".ljust(70) + " ║")
        print("║" + " "*70 + "║")
        print(f"║  Spinup from rest: {result['spinup_time_s']*1000:.0f} ms".ljust(70) + " ║")
        print(f"║  RPM drop per {roller_width}-ball burst: {result['rpm_drop_pct']:.1f}%".ljust(70) + " ║")
        print(f"║  Recovery time: {result['recovery_time_s']*1000:.0f} ms".ljust(70) + " ║")
        print("║" + " "*70 + "║")
        print(f"║  ★ RATE OF FIRE: {result['balls_per_second']:.1f} balls/second ★".ljust(70) + " ║")
        print(f"║    ({result['bursts_per_second']:.1f} bursts/second × {roller_width} balls)".ljust(70) + " ║")
    else:
        print(f"║  ✗ NOT FEASIBLE: {result['reason']}".ljust(70) + " ║")
    
    print("║" + " "*70 + "║")
    print("║ TRAJECTORY COVERAGE".ljust(70) + " ║")
    print("║" + "-"*70 + "║")
    print(f"║  Release height: {release_height}\"".ljust(70) + " ║")
    
    if trajectory_coverage:
        min_dist = trajectory_coverage[0][0]
        max_dist = trajectory_coverage[-1][0]
        min_entry = min(tc[2] for tc in trajectory_coverage)
        max_entry = max(tc[2] for tc in trajectory_coverage)
        print(f"║  Range: {min_dist} ft to {max_dist} ft".ljust(70) + " ║")
        print(f"║  Entry angles: {min_entry:.0f}° to {max_entry:.0f}° (all descending)".ljust(70) + " ║")
        
        ideal_count = sum(1 for tc in trajectory_coverage if tc[2] >= 60)
        good_count = sum(1 for tc in trajectory_coverage if 45 <= tc[2] < 60)
        print(f"║  Quality: {ideal_count} IDEAL (≥60°), {good_count} GOOD (45-60°)".ljust(70) + " ║")
    else:
        print("║  ✗ No valid lob trajectories at this velocity".ljust(70) + " ║")
    
    print("╚" + "═"*70 + "╝")
    
    # Quick reference table
    if trajectory_coverage:
        print("\n┌" + "─"*50 + "┐")
        print("│" + " QUICK REFERENCE TABLE ".center(50) + "│")
        print("├" + "─"*50 + "┤")
        print("│ {:>8} │ {:>12} │ {:>12} │ {:>8} │".format("Distance", "Launch Angle", "Entry Angle", "Quality"))
        print("├" + "─"*50 + "┤")
        for d_ft, launch, entry in trajectory_coverage:
            quality = "IDEAL" if entry >= 60 else "GOOD" if entry >= 45 else "RISK"
            print("│ {:>6} ft │ {:>10.0f}° │ {:>10.0f}° │ {:>8} │".format(d_ft, launch, entry, quality))
        print("└" + "─"*50 + "┘")

# Interactive version
interact(generate_design_summary,
         motor_type=Dropdown(options=['NEO', 'FALCON500', 'KRAKENX60'], value='KRAKENX60', description='Motor'),
         num_motors=IntSlider(min=1, max=4, value=2, description='# Motors'),
         gear_ratio=FloatSlider(min=0.5, max=3.0, step=0.1, value=1.5, description='Gear Ratio'),
         roller_diameter=FloatSlider(min=3.0, max=6.0, step=0.5, value=4.0, description='Roller (in)'),
         roller_width=IntSlider(min=1, max=4, value=2, description='Width'),
         added_flywheel=FloatSlider(min=0, max=30, step=2, value=0, description='+Flywheel'),
         target_velocity=IntSlider(min=25, max=40, value=30, description='Target (fps)'),
         release_height=IntSlider(min=14, max=22, value=18, description='Release (in)'));